Connecting Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Installing Required Libraries

In [2]:
!pip install moviepy

!pip3 install imageio==2.4.1

!pip install --upgrade imageio-ffmpeg

!pip install tensorflow --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 69.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for imageio: filename=imageio-2.4.1-py3-none-any.whl size=3303886 sha256=7d7a55759a050cae02fb6aca45936899563f53fe0f638b474f9d385caabdc561
  Stored in directory: /root/.cache/pip/wheels/b7/44/b7/2e7cc9c5fe4a893b9cc83a010d4410557bedf6cf3b5829f497
Successfully built imageio
  Attempting uninstall: imageio
    Found existing installation: imageio 2.25.1
    Uninstalling imageio-2.25.1:
      Successfully uninstalled imageio-2.25.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
moviepy 1.0.3 requires imageio<3.0,>=2.5; python_version >= 

Importing Libraries

In [3]:
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2,preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image
import numpy as np
import pandas as pd
import cv2
import os
from scipy import spatial
from imutils.video import FileVideoStream
import imutils
import tensorflow as tf
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
from moviepy.editor import *
import matplotlib.pyplot as plt
import cv2
import subprocess
from moviepy.config import get_setting
from moviepy.tools import subprocess_call
import numba
import threading
import cupy as cp
import logging;
logger = logging.getLogger("numba");
logger.setLevel(logging.ERROR)

In [4]:
def return_image_embedding1(model,img_path):
    img = image.load_img(img_path)
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    preds = model.predict(x)
    return np.array(preds).flatten()
    #return preds

In [5]:
def return_image_embedding2(model,img):
    # img = np.resize(frame, (299,299,3))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    preds = model.predict(x)
    return preds

Removing Frame Folders

In [22]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"
%rm -rf /content/drive/MyDrive/MID/FootballData/Camera_Angle1
%rm -rf /content/drive/MyDrive/MID/FootballData/Camera_Angle2
%rm -rf /content/drive/MyDrive/MID/FootballData/Videos

Creating Frame Folders

In [7]:
import os.path
from os import path
path = "/content/drive/MyDrive/MID/FootballData/Camera_Angle1"
os.mkdir(path)
path = "/content/drive/MyDrive/MID/FootballData/Camera_Angle2"
os.mkdir(path)
path = "/content/drive/MyDrive/MID/FootballData/Videos"
os.mkdir(path)

In [8]:
model = InceptionResNetV2(include_top=False,weights='imagenet',pooling='avg')

219055592/219055592 [==============================] - 8s 0us/step


In [9]:
def cp_hist_to_cv(np_histogram_output):
    counts, bin_edges = np_histogram_output
    counts = cp.asnumpy(counts)
    return counts.ravel().astype('float32')

FPS Extraction

In [10]:
videopath = "/content/drive/MyDrive/MID/12.mp4"
video = cv2.VideoCapture(videopath);
fps = video.get(cv2.CAP_PROP_FPS)
print(fps)

25.0


In [11]:
from imutils.video import FileVideoStream
import imutils
import time
import numpy as np
import cupy as cp
import cv2
import math

vs = FileVideoStream(videopath).start()

frame1 = vs.read()

j = 0

diction1 = {}
diction2 = {}

a = 0
ls1 = []
ls2 = []

while True:


  if not vs.more():
      vs.stop()
      break

  a += 1

  frame2 = vs.read()
  
  if frame2 is not None:

    hist_img1 = cp.histogram(cp.array(frame1))
    # np.normalize(hist_img1, hist_img1, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX)
    hist_img2 = cp.histogram(cp.array(frame2))
    # cv2.normalize(hist_img2, hist_img2, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX)

    # metric_val = bhattacharyya(cp_hist_to_cv(hist_img1),cp_hist_to_cv(hist_img2))
    metric_val = cv2.compareHist(cp_hist_to_cv(hist_img1), cp_hist_to_cv(hist_img2),
                        cv2.HISTCMP_HELLINGER)

    # print(metric_val)
    if metric_val >= 0.13:
      # k = 1
      # cv2.imwrite("/content/drive/MyDrive/MID/FootballData/Camera Angel/" + str(j+1) +  str(k) +".jpeg",frame1)
      # k = 2
      # cv2.imwrite("/content/drive/MyDrive/MID/FootballData/Camera Angel/" + str(j+1) + str(k) + ".jpeg",frame2)
      ls1.append(frame1)
      ls1.append(a/fps)
      ls2.append(frame2)
      ls2.append(a/fps)
      diction1["/content/drive/MyDrive/MID/FootballData/Camera_Angle1/" + str(j+1) + ".jpeg"] = ls1
      diction2["/content/drive/MyDrive/MID/FootballData/Camera_Angle2/" + str(j+1) + ".jpeg"] = ls2

 
      ls1 = []
      ls2 = []


      j += 1


    if a % 10000 == 0:
      print(a)

    frame1 = frame2


10000


In [12]:
print(len(diction1))
print(len(diction2))

106
106


Testing

In [13]:
# for i in diction1.keys():
#   cv2.imwrite("1.jpeg",diction1[i][0])

In [14]:
from time import sleep
import matplotlib.pyplot as plt

for i in diction1.keys():
  cv2.imwrite(i,diction1[i][0])
  cv2.imwrite(i.replace("Angle1","Angle2"),diction2[i.replace("Angle1","Angle2")][0])

Finding Relevant Frames

In [15]:
import os
path1 = "/content/drive/MyDrive/MID/FootballData/Camera_Angle1/"
path2 = "/content/drive/MyDrive/MID/FootballData/Camera_Angle2/"
count=0
files = sorted(os.listdir("/content/drive/MyDrive/MID/FootballData/Camera_Angle1"))

frametest2 = return_image_embedding1(model,"/content/drive/MyDrive/Testing/2.jpeg")
frametest3 = return_image_embedding1(model,"/content/drive/MyDrive/Testing/3.jpeg")
frametest4 = return_image_embedding1(model,"/content/drive/MyDrive/Testing/4.jpeg")
frametest7 = return_image_embedding1(model,"/content/drive/MyDrive/Testing/7.png")#Penalty Kick side facing camera
frametest8= return_image_embedding1(model,"/content/drive/MyDrive/Testing/8.png")

p1 = return_image_embedding1(model,"/content/drive/MyDrive/Testing/P1.png")
p2 = return_image_embedding1(model,"/content/drive/MyDrive/Testing/P2.png")
card1=return_image_embedding1(model,"/content/drive/MyDrive/Testing/C1.png")


ls1 = []
for i in files:
  frame1 = return_image_embedding1(model,path1 + i)
  frame2 = return_image_embedding1(model,path2 + i)

  similarity12 = -1 * (spatial.distance.cosine(frametest2, frame1) - 1)
  similarity13 = -1 * (spatial.distance.cosine(frametest3, frame1) - 1)
  similarity14 = -1 * (spatial.distance.cosine(frametest4, frame1) - 1)
  
  similarity17 = -1 * (spatial.distance.cosine(frametest4, frame1) - 1)
  similarity18 = -1 * (spatial.distance.cosine(frametest8, frame1) - 1)
  c1 = -1 * (spatial.distance.cosine(card1, frame1) - 1)
  p1_s= -1 * (spatial.distance.cosine(p1, frame1) - 1)

  similarity27 = -1 * (spatial.distance.cosine(frametest7, frame2) - 1)
  
  p2_s= -1 * (spatial.distance.cosine(p2, frame2) - 1)
  # print("Penalty P1: ",p1_s,   "-----------"  ,i)
  # print("Penalty P2: ",p2_s,   "-----------"  ,i)
  # print("Pic1\t",similarity12,   "-----------"  ,i)
  # print("Pic2\t",similarity13,   "-----------"  ,i)
  # print("Pic3\t",similarity14,   "-----------"  ,i)
  # print("Pic4\t",similarity17,   "-----------"  ,i)
  # print("Pic5\t",similarity18,   "-----------"  ,i)
  
  # print("Pic6:\t",similarity27,   "-----------"  ,i)
  #print("Card:\t",c1,   "-----------"  ,i)


  """
For Penalties
 if (p1_s >= 0.935 or p2_s >=0.936):

Normal Condition
if (similarity12 >= 0.935 or similarity13 >= 0.935  or similarity17 >= 0.950
      or similarity27 >= 0.950 or p1_s >= 0.935 or p2_s >=0.936):

No Penalties but everything else
if ( similarity12 >= 0.935 or similarity13 >= 0.935
  or similarity17 >= 0.950 or similarity27 >= 0.950):



if (similarity12 >= 0.922 or similarity13 >= 0.935
  or (similarity17 >= 0.95 or similarity18 >= 0.96)
   or (p1_s >= 0.935 or p2_s >=0.935) or c1 >= 0.925):

"""


  if (similarity12 >= 0.922 or similarity13 >= 0.935
  or (similarity17 >= 0.95 or similarity18 >= 0.96)
   or (p1_s >= 0.935 or p2_s >=0.935) or c1 >= 0.925):
    ls1.append(i)
    count+=1
    print("Penalty P1: ",p1_s,   "-----------"  ,i)
    print("Penalty P2: ",p2_s,   "-----------"  ,i)
    print("Pic1\t",similarity12,   "-----------"  ,i)
    print("Pic2\t",similarity13,   "-----------"  ,i)
    print("Pic3\t",similarity14,   "-----------"  ,i)
    print("Pic4\t",similarity17,   "-----------"  ,i)
    print("Pic5\t",similarity18,   "-----------"  ,i)
    print("Card:\t",c1,   "-----------"  ,i)
    print("Pic6:\t",similarity27,   "-----------"  ,i)
    
   
    print()

print(count)      

1/1 [==============================] - 0s 70ms/step
Penalty P1:  0.9216880798339844 ----------- 42.jpeg
Penalty P2:  0.8428342938423157 ----------- 42.jpeg
Pic1	 0.9275664687156677 ----------- 42.jpeg
Pic2	 0.9115633964538574 ----------- 42.jpeg
Pic3	 0.8861569166183472 ----------- 42.jpeg
Pic4	 0.8861569166183472 ----------- 42.jpeg
Pic5	 0.9651273488998413 ----------- 42.jpeg
Card:	 0.5060946345329285 ----------- 42.jpeg
Pic6:	 0.8843939900398254 ----------- 42.jpeg

1/1 [==============================] - 0s 69ms/step
Penalty P1:  0.612619161605835 ----------- 51.jpeg
Penalty P2:  0.6765782833099365 ----------- 51.jpeg
Pic1	 0.5175154209136963 ----------- 51.jpeg
Pic2	 0.494180291891098 ----------- 51.jpeg
Pic3	 0.5302484631538391 ----------- 51.jpeg
Pic4	 0.5302484631538391 ----------- 51.jpeg
Pic5	 0.48531848192214966 ----------- 51.jpeg
Card:	 0.9548909664154053 ----------- 51.jpeg
Pic6:	 0.6626294851303101 ----------- 51.jpeg

1/1 [==============================] - 0s 69ms/step
P

Removing Unnecessary Frames

In [16]:
for i in os.listdir("/content/drive/MyDrive/MID/FootballData/Camera_Angle1/"):
  if i not in ls1:
    os.remove("/content/drive/MyDrive/MID/FootballData/Camera_Angle1/" + i)

In [17]:
video_dictionary = {}
for i in os.listdir("/content/drive/MyDrive/MID/FootballData/Camera_Angle1/"):
  video_dictionary["/content/drive/MyDrive/MID/FootballData/Videos/" + i.replace(".jpeg",".mp4")] = diction1["/content/drive/MyDrive/MID/FootballData/Camera_Angle1/" + i] 

Creating Frames into Clips

In [18]:
# cutting out some part from the clip
for i in video_dictionary.keys():
  cmd = [get_setting("FFMPEG_BINARY"),"-y",
          "-ss", "%0.2f"%(video_dictionary[i][1]-6),
          "-i", videopath,
          "-t", "%0.2f"%(18),
          "-map", "0", "-vcodec", "copy", "-acodec", "copy", i]
  
  subprocess_call(cmd)

Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful


In [19]:
print(len(video_dictionary))

6


Results Derived Here

In [20]:
from moviepy.editor import *
import os
s=str(len(os.listdir("/content/drive/MyDrive/MID/FootballData/Results")))


c = []
for i in os.listdir("/content/drive/MyDrive/MID/FootballData/Videos"):
  c.append(VideoFileClip("/content/drive/MyDrive/MID/FootballData/Videos/" + i))

final_clip = concatenate_videoclips(c, method="compose")
final_clip.write_videofile("/content/drive/MyDrive/MID/FootballData/Results/result"+s+".mp4")

Moviepy - Building video /content/drive/MyDrive/MID/FootballData/Results/result1.mp4.
MoviePy - Writing audio in result1TEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video /content/drive/MyDrive/MID/FootballData/Results/result1.mp4



t:  17%|█▋        | 451/2728 [01:36<09:03,  4.19it/s, now=None]WARNING:py.warnings:/usr/local/lib/python3.9/dist-packages/moviepy/video/io/ffmpeg_reader.py:123: UserWarning: Warning: in file /content/drive/MyDrive/MID/FootballData/Videos/7.mp4, 6220800 bytes wanted but 0 bytes read,at frame 450/456, at time 18.00/18.21 sec. Using the last valid frame instead.
  warnings.warn("Warning: in file %s, "%(self.filename)+

t:  17%|█▋        | 452/2728 [01:36<07:55,  4.79it/s, now=None]WARNING:py.warnings:/usr/local/lib/python3.9/dist-packages/moviepy/video/io/ffmpeg_reader.py:123: UserWarning: Warning: in file /content/drive/MyDrive/MID/FootballData/Videos/7.mp4, 6220800 bytes wanted but 0 bytes read,at frame 451/456, at time 18.04/18.21 sec. Using the last valid frame instead.
  warnings.warn("Warning: in file %s, "%(self.filename)+

t:  17%|█▋        | 453/2728 [01:36<07:47,  4.87it/s, now=None]WARNING:py.warnings:/usr/local/lib/python3.9/dist-packages/moviepy/video/io/ffmpeg_reader.py:123:

Moviepy - Done !
Moviepy - video ready /content/drive/MyDrive/MID/FootballData/Results/result1.mp4
